Check coefficients for integration schemes - they should all line up nicely for values in the middle and vary smoothly

In [28]:
from bokeh import plotting, io, models, palettes
io.output_notebook()

import numpy
from maxr.integrator import history

nmax = 5
figures = []
palette = palettes.Category10[3]
for n in range(1, nmax):
    fig = plotting.figure(height=100, width=600,
                          active_drag='pan', active_scroll='wheel_zoom')
    for order, color in zip((1, 2, 3), palette):
        try:
            coeffs = history.coefficients(n, order=order)
            ticks = range(len(coeffs))
            fig.line(ticks, coeffs, alpha=0.9, color=color)
            fig.circle(ticks, coeffs, alpha=0.9, color=color)
        except ValueError:
            # Skip orders if we don't have enough coefficients to calculate these
            continue
    fig.yaxis.axis_label = 'n={0}'.format(n)
    fig.toolbar.logo = None
    fig.toolbar_location = None
    figures.append(fig)
    
    # Set up scaling
    if len(figures) == 1:
        figures[0].x_range = models.Range1d(0, nmax - 1)
        figures[0].y_range = models.Range1d(0, 2)
    else:
        figures[-1].x_range = figures[0].x_range
        figures[-1].y_range = figures[0].y_range

io.show(models.Column(*figures))

Loading BokehJS ...

Define some timesteps to integrate over

In [29]:
tmin, tmax = 0, 30
ts = numpy.linspace(tmin, tmax, 1000)

Check we can integrate things!

In [30]:
expected = -1.2492166377597749

In [31]:
history.integrator(numpy.sin(ts), ts) - expected < 1e-5

True

Turn this into a history integrator for a python function

In [84]:
def evaluate_history_integral(f, ts, order=1):
    """ Evaluate the history integral for a given driving function f
    """
    return numpy.array([0] + [
        history.integrator(f(ts[:idx+1]), ts[:idx+1], order=order)
        for idx in range(1, len(ts))])

In [93]:
results = evaluate_history_integral(numpy.sin, ts)

figure = plotting.figure(height=300)
figure.line(ts, results)
figure.title.text = "∫sin(t)/√(t-𝜏)d𝜏"
io.show(figure)

Check accuracy of convergence. We use a sinusoidal forcing and plot the response
$$ 
\int_0^{t} \frac{\sin{(\tau)}}{\sqrt{t - \tau}}d\tau = \sqrt{2 \pi}\left[C{\left(\sqrt{\frac{2t}{\pi}}\right)}\sin{t} - S{\left(\sqrt{\frac{2t}{\pi}}\right)}\cos{t}\right]
$$
where $C$ is the Fresnel C (cos) integral, and $S$ is the Fresnel $S$ (sin) integral. Note the solution in the paper is **WRONG**

In [82]:
from scipy.special import fresnel

def solution(t):
    ssc, csc = fresnel(numpy.sqrt(2 * t / numpy.pi)) 
    return numpy.sqrt(2 * numpy.pi) * (
        csc * numpy.sin(t) - ssc * numpy.cos(t))

Show the solution

In [87]:
figure = plotting.figure(height=300)
figure.line(ts, numpy.sin(ts), legend='Source function sin(t)', color=palette[1], alpha=0.7)
figure.line(ts, solution(ts), legend='Analytic ∫sin(t)/√(t-𝜏)d𝜏', color=palette[0], alpha=0.7)
figure.line(ts, evaluate_history_integral(numpy.sin, ts), legend='Numerical ∫sin(t)/√(t-𝜏)d𝜏', color=palette[2], alpha=0.7)
io.show(figure)

and try integration numerically

In [88]:
nsteps = 30
order = 3
tmin = 0
tmax = 40

# Evaluate solution 
ts = numpy.linspace(tmin, tmax, nsteps)
numeric = evaluate_history_integral(numpy.sin, ts, order=order)
exact = solution(ts)

figure = plotting.figure(height=300)
figure.line(ts, exact, legend='Analytic', color=palette[0], alpha=0.7)
figure.line(ts, numeric, legend='Numerical', color=palette[2], alpha=0.7)
io.show(figure)

In [89]:
numpy.mean(numeric - exact)

0.015557324193794015

Now we loop through by order and computer the error

In [90]:
from collections import defaultdict

# Set up steps
nstepstep = 25
nsteps = numpy.arange(nstepstep, 500, nstepstep)
spacing = 10 / (nsteps - 1)

# Calculate error
error = defaultdict(list)
for order in (1, 2, 3):
    for N in nsteps:
        ts = numpy.linspace(0, tmax, N)
        err = evaluate_history_integral(numpy.sin, ts, order=order) - solution(ts)
        error[order].append(abs(err).max())

# Convert to arrays
for key, value in error.items():
    error[key] = numpy.asarray(value)

We can plot how the error changes with spacing

In [91]:
figure = plotting.figure(height=300, x_axis_type='log', y_axis_type='log')
for order, color in zip((1, 2, 3), palette):
    figure.line(spacing, error[order], legend='Order = {0}'.format(order),
                color=color, alpha=0.9)
figure.xaxis.axis_label = 'Timestep (𝛿t)'
figure.yaxis.axis_label = 'Error (𝜀)'
figure.legend.location = 'bottom_right'
io.show(figure)

check that we get reasonable scaling (should be about $\epsilon\sim\delta t ^{\text{order} + 1}$)

In [92]:
def slope(rise, run):
    return (rise[1:] - rise[0]) / (run[1:] - run[0])

figure = plotting.figure(height=300, x_axis_type='log')
for order, color in zip((1, 2, 3), palette):
    figure.line(spacing[1:], 
                slope(numpy.log(error[order]), numpy.log(spacing)), 
                legend='Order = {0}'.format(order),
                color=color, alpha=0.9)
figure.xaxis.axis_label = 'Timestep (𝛿t)'
figure.yaxis.axis_label = 'Scaling exponent'
figure.legend.location = 'center_right'
io.show(figure)